# Capítulo 3

## Dataframes

In [ ]:

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._

/**
  * Usage: MnMcount <mnm_file_dataset>
  */

val spark = SparkSession
      .builder
      .appName("MnMCount")
      .getOrCreate()

    // read the file into a Spark DataFrame
val mnmDF = spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load("C:/Users/xabier.jimenez/Documents/data/mnm_dataset.csv")
    // display DataFrame
mnmDF.show(5, false)

    // aggregate count of all colors and groupBy state and color
    // orderBy descending order
val countMnMDF = mnmDF.select("State", "Color", "Count")
        .groupBy("State", "Color")
        .sum("Count")
        .orderBy(desc("sum(Count)"))

    // show all the resulting aggregation for all the dates and colors
countMnMDF.show(60)
println("Total Rows = ${countMnMDF.count()}")
println()

    // find the aggregate count for California by filtering
val caCountMnNDF = mnmDF.select("*")
      .where(col("State") === "CA")
      .groupBy("State", "Color")
      .sum("Count")
      .orderBy(desc("sum(Count)"))

    // show the resulting aggregation for California
caCountMnNDF.show(10)
  


Del ejercicio de M&M aplicar: 
i. Otras operaciones de agregación como el Max con otro tipo de 
ordenamiento (descendiente)

In [ ]:
val caMaxMnNDF = mnmDF.select("*")
      .where(col("State") === "CA")
      .groupBy("State", "Color")
      .max("Count")
      .orderBy(desc("max(Count)"))
caMaxMnNDF.show(10)

ii. hacer un ejercicio como el “where” de CA que aparece en el libro pero 
indicando más opciones de estados (p.e. NV, TX, CA, CO).

In [ ]:
val nvCountMnNDF = mnmDF.select("*")
      .where(col("State") === "NV")
      .groupBy("State", "Color")
      .sum("Count")
      .orderBy(desc("sum(Count)"))
nvCountMnNDF.show(10)

val txCountMnNDF = mnmDF.select("*")
      .where(col("State") === "TX")
      .groupBy("State", "Color")
      .sum("Count")
      .orderBy(desc("sum(Count)"))
txCountMnNDF.show(10)

val coCountMnNDF = mnmDF.select("*")
      .where(col("State") === "CO")
      .groupBy("State", "Color")
      .sum("Count")
      .orderBy(desc("sum(Count)"))
coCountMnNDF.show(10)

iii. Hacer un ejercicio donde se calculen en una misma operación el Max, 
Min, Avg, Count. Revisar el API (documentación) donde encontrarán 
este ejemplo:
ds.agg(max($"age"), avg($"salary"))
ds.groupBy().agg(max($"age"), avg($"salary"))
NOTA: $ es un alias de col()

In [ ]:
val countMnMDF = mnmDF
 .select("State", "Color", "Count")
 .groupBy("State", "Color")
 .agg(count("Count").alias("Total"),max($"Count"),min($"Count"), avg($"Count"))
 .orderBy(desc("Total"))

countMnMDF.show(60)

iv. Hacer también ejercicios en SQL creando tmpVie

In [ ]:
mnmDF.createOrReplaceTempView("mnmDFtemp")

spark.sql("""SELECT * 
FROM mnmDFtemp 
WHERE Color='Blue'""").show(10)

# AuthorAges

In [ ]:
import org.apache.spark.sql.functions.avg
import org.apache.spark.sql.SparkSession
// Create a DataFrame using SparkSession
val spark = SparkSession
 .builder
 .appName("AuthorsAges")
 .getOrCreate()
// Create a DataFrame of names and ages
val dataDF = spark.createDataFrame(Seq(("Brooke", 20), ("Brooke", 25),
 ("Denny", 31), ("Jules", 30), ("TD", 35))).toDF("name", "age")
// Group the same names together, aggregate their ages, and compute an average
val avgDF = dataDF.groupBy("name").agg(avg("age"))
avgDF.show()


In [ ]:
import org.apache.spark.sql.types._
val schema = StructType(Array(StructField("author", StringType, false),
 StructField("title", StringType, false),
 StructField("pages", IntegerType, false)))

In [ ]:
val spark = SparkSession
 .builder
 .appName("Example-3_7")
 .getOrCreate()



val jsonFile = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/chapter3/data/blogs.json"
 // Define our schema programmatically
 val schema = StructType(Array(StructField("Id", IntegerType, false),
 StructField("First", StringType, false),
 StructField("Last", StringType, false),
 StructField("Url", StringType, false),
 StructField("Published", StringType, false),
 StructField("Hits", IntegerType, false),
 StructField("Campaigns", ArrayType(StringType), false)))
 // Create a DataFrame by reading from the JSON file 
 // with a predefined schema
 val blogsDF = spark.read.schema(schema).json(jsonFile)
 // Show the DataFrame schema as output
 blogsDF.show(false)
println(blogsDF.printSchema)
println(blogsDF.schema)


In [ ]:
blogsDF.withColumn("Big Hitters", (expr("Hits > 10000"))).show()

In [ ]:
blogsDF
 .withColumn("AuthorsId", (concat(expr("First"), expr("Last"), expr("Id"))))
 .select(col("AuthorsId"))
 .show(4)

In [ ]:
blogsDF.select(expr("Hits")).show(2)
blogsDF.select(col("Hits")).show(2)
blogsDF.select("Hits").show(2)

In [ ]:
blogsDF.sort(col("Id").desc).show()
blogsDF.sort($"Id".desc).show()

In [ ]:
import org.apache.spark.sql.Row
// Create a Row
val blogRow = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015",
 Array("twitter", "LinkedIn"))
// Access using index for individual items
blogRow(1)


In [ ]:
val rows = Seq(("Matei Zaharia", "CA"), ("Reynold Xin", "CA"))
val authorsDF = rows.toDF("Author", "State")
authorsDF.show()


In [ ]:
val sampleDF = spark
 .read
 .option("samplingRatio", 0.001)
 .option("header", true)
 .csv("""C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/chapter3/data/sf-fire-calls.csv""")


In [ ]:
val fireSchema =  StructType(Array(StructField("CallNumber", IntegerType, true),
 StructField("UnitID", StringType, true),
 StructField("IncidentNumber", IntegerType, true),
 StructField("CallType", StringType, true),
 StructField("CallDate", StringType, true), 
 StructField("WatchDate", StringType, true),
 StructField("CallFinalDisposition", StringType, true),
 StructField("AvailableDtTm", StringType, true),
 StructField("Address", StringType, true), 
 StructField("City", StringType, true), 
 StructField("Zipcode", IntegerType, true), 
 StructField("Battalion", StringType, true), 
 StructField("StationArea", StringType, true), 
 StructField("Box", StringType, true), 
 StructField("OriginalPriority", StringType, true), 
 StructField("Priority", StringType, true), 
 StructField("FinalPriority", IntegerType, true), 
 StructField("ALSUnit", BooleanType, true), 
 StructField("CallTypeGroup", StringType, true),
 StructField("NumAlarms", IntegerType, true),
 StructField("UnitType", StringType, true),
 StructField("UnitSequenceInCallDispatch", IntegerType, true),
 StructField("FirePreventionDistrict", StringType, true),
 StructField("SupervisorDistrict", StringType, true),
 StructField("Neighborhood", StringType, true),
 StructField("Location", StringType, true),
 StructField("RowID", StringType, true),
 StructField("Delay", FloatType, true)))
// Read the file using the CSV DataFrameReader
val sfFireFile="C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/chapter3/data/sf-fire-calls.csv"
val fireDF = spark.read.schema(fireSchema)
 .option("header", "true")
 .csv(sfFireFile)


In [ ]:
val parquetPath = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/chapter3/parquet/fire_df.parquet"
fireDF.repartition(1).write.mode("overwrite").format("parquet").save(parquetPath)
val parquetTable ="C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/chapter3/parquet/fire_df.parquet"
fireDF.repartition(1).write.mode("overwrite").format("parquet").saveAsTable(parquetTable)

In [ ]:
val fewFireDF = fireDF
 .select("IncidentNumber", "AvailableDtTm", "CallType")
 .where($"CallType" =!= "Medical Incident") 
fewFireDF.show(5, false)


In [ ]:
fireDF
 .select("CallType")
 .distinct()
 .where($"CallType".isNotNull)
 .show(10, false)


In [ ]:
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")
newFireDF
 .select("ResponseDelayedinMins")
 .where($"ResponseDelayedinMins" > 5)
 .show(5, false)

In [ ]:
val fireTsDF = newFireDF
 .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy"))
 .drop("CallDate")
 .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy"))
 .drop("WatchDate")
 .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"),
 "MM/dd/yyyy hh:mm:ss a"))
 .drop("AvailableDtTm")
// Select the converted columns
fireTsDF
 .select("IncidentDate", "OnWatchDate", "AvailableDtTS")
 .show(5, false)

In [ ]:
fireTsDF
 .select(year($"IncidentDate"))
 .distinct()
 .orderBy(year($"IncidentDate"))
 .show()


In [ ]:
fireTsDF
 .select("CallType")
 .where(col("CallType").isNotNull)
 .groupBy("CallType")
 .count()
 .orderBy(desc("count"))
 .show(10, false)

In [ ]:
import org.apache.spark.sql.{functions => F}
fireTsDF
 .select(F.sum("NumAlarms"), F.avg("ResponseDelayedinMins"),
 F.min("ResponseDelayedinMins"), F.max("ResponseDelayedinMins"))
 .show()

## Datasets

In [ ]:
import org.apache.spark.sql.Row
val row = Row(350, true, "Learning Spark 2E", null)
row.getInt(0)
row.getBoolean(1)
row.getString(2)

In [ ]:
case class DeviceIoTData (battery_level: Long, c02_level: Long,
cca2: String, cca3: String, cn: String, device_id: Long,
device_name: String, humidity: Long, ip: String, latitude: Double,
lcd: String, longitude: Double, scale:String, temp: Long,
timestamp: Long)
import spark.implicits._
import org.apache.spark.sql.Encoders
val ds = spark.read
.json("C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/iot-devices/iot_devices.json")
.as[DeviceIoTData]
ds.show(5, false)

In [ ]:
//val filterTempDS = ds.filter({d => {d.temp > 30 && d.humidity > 70})
val filterTempDS = ds.filter(ds("temp") > 30 && ds("humidity") > 70)
filterTempDS.show(5, false)

In [ ]:
case class DeviceTempByCountry(temp: Long, device_name: String, device_id: Long,
 cca3: String)

In [ ]:
val dsTemp = ds
 .filter(ds("temp") > 25)
 .map(ds("temp"), ds("device_name"), ds("device_id"), ds("cca3"))
 .toDF("temp", "device_name", "device_id", "cca3").as[DeviceTempByCountry]
dsTemp.show(5, false)

In [ ]:
val device = dsTemp.first()
println(device)

In [ ]:
val dsTemp2 = ds
 .select($"temp", $"device_name", $"device_id", $"device_id", $"cca3")
 .where("temp > 25")
 .as[DeviceTempByCountry]

# Ejercicios capítulo 3

Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por 
defecto.

In [ ]:
val mnmDF = spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true")
      .load("C:/Users/xabier.jimenez/Documents/data/mnm_dataset.csv")
    // display DataFrame
mnmDF.show(5, false)

Cuando se define un schema al definir un campo por ejemplo StructField('Delay', 
FloatType(), True) ¿qué significa el último parámetro Boolean?

La posibilidad de contener nulos

Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?

El dataset lo hemos definido como una clase DeviceIoTData

Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y 
guardar los datos en los formatos:
i. JSON
ii. CSV (dándole otro nombre para evitar sobrescribir el fichero origen)
iii. AVRO

In [ ]:
mnmDF.write
 .mode("overwrite")
 .saveAsTable("mnm")

In [ ]:
mnmDF.write.json("C:/tmp/data/json/df_mnm")

In [ ]:
mnmDF.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_mnm")

In [ ]:
mnmDF.write
 .format("avro")
 .mode("overwrite")
 .save("/tmp/data/avro/dfmnm_avro")

Revisar al guardar los ficheros (p.e. json, csv, etc) el número de ficheros 
creados, revisar su contenido para comprender (constatar) como se guardan.
i. ¿A qué se debe que hayan más de un fichero?
ii. ¿Cómo obtener el número de particiones de un DataFrame?
iii. ¿Qué formas existen para modificar el número de particiones de un 
DataFrame?
iv. Llevar a cabo el ejemplo modificando el número de particiones a 1 y 
revisar de nuevo el/los ficheros guardados. 

Hay más de un fichero porque lo particiona

In [ ]:
mnmDF.rdd.getNumPartitions

Para modificarlo se usa .repartition()

In [ ]:
mnmDF.repartition(1).write
 .mode("overwrite")
 .saveAsTable("mnm")

val df = spark.read.format("json")
.load("LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")
df.coalesce(1).write.json("C:/tmp/data/json/df_json_1_part")

## Capítulo 4

In [ ]:
import org.apache.spark.sql.SparkSession 
val spark = SparkSession
.builder
.enableHiveSupport()
.appName("SparkSQLExampleApp")
.getOrCreate()
// Path to data set 
val csvFile="C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
// Read and create a temporary view
// Infer schema (note that for larger files you may want to specify the schema)
val df = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .load(csvFile)
// Create a temporary view
df.createOrReplaceTempView("us_delay_flights_tbl")

In [ ]:
spark.stop()

// En scala hay inicializar sparksession, parar(stop) e iniciar otra vez

In [ ]:
val schema = "date STRING, delay INT, distance INT, origin STRING, destination STRING"

In [ ]:
spark.sql("""SELECT distance, origin, destination 
FROM us_delay_flights_tbl WHERE distance > 1000 
ORDER BY distance DESC""").show(10)

In [ ]:
spark.sql("""SELECT date, delay, origin, destination 
FROM us_delay_flights_tbl 
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD' 
ORDER by delay DESC""").show(10)

In [ ]:
spark.sql("""SELECT delay, origin, destination,
 CASE
 WHEN delay > 360 THEN 'Very Long Delays'
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays'
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays'
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
 WHEN delay = 0 THEN 'No Delays'
 ELSE 'Early'
 END AS Flight_Delays
 FROM us_delay_flights_tbl
 ORDER BY origin, delay DESC""").show(10)

In [ ]:
spark.sql("CREATE DATABASE learn_spark_db")
spark.sql("USE learn_spark_db")

In [ ]:
spark.sql("CREATE TABLE if not exists learn_spark_db.managed_us_delay_flights_tbl5 (date STRING, delay INT, distance INT, origin STRING, destination STRING)")

In [ ]:
spark.sql("show tables").show(false)

In [ ]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, 
 distance INT, origin STRING, destination STRING) 
 USING csv OPTIONS (PATH 
 'C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')""")

In [ ]:
val df_sfo = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'SFO'")
val df_jfk = spark.sql("SELECT date, delay, origin, destination FROM us_delay_flights_tbl WHERE origin = 'JFK'")
//Create a temporary and global temporary view
df_sfo.createOrReplaceGlobalTempView("us_origin_airport_SFO_global_tmp_view")
df_jfk.createOrReplaceTempView("us_origin_airport_JFK_tmp_view")
spark.sql("SELECT * FROM us_origin_airport_JFK_tmp_view")

In [ ]:
spark.catalog.dropGlobalTempView("us_origin_airport_SFO_global_tmp_view")
spark.catalog.dropTempView("us_origin_airport_JFK_tmp_view")

In [ ]:
spark.catalog.listDatabases()

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.catalog.listColumns("us_delay_flights_tbl")

In [ ]:
val usFlightsDF = spark.sql("SELECT * FROM us_delay_flights_tbl")
val usFlightsDF2 = spark.table("us_delay_flights_tbl")

## data Sources for DataFrames and SQL Tables

In [ ]:
// Use Parquet 
val file = """C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet"""
val df = spark.read.format("parquet").load(file)
// Use Parquet; you can omit format("parquet") if you wish as it's the default
val df2 = spark.read.load(file)
// Use CSV
val df3 = spark.read.format("csv")
 .option("inferSchema", "true")
 .option("header", "true")
 .option("mode", "PERMISSIVE")
 .load("C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*")
// Use JSON
val df4 = spark.read.format("json")
 .load("C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*")

In [ ]:
val file = """C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/parquet/2010-summary.parquet/"""
val df = spark.read.format("parquet").load(file)

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

In [ ]:
df.write.format("parquet")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/parquet/df_parquet")

In [ ]:
df.write
 .mode("overwrite")
 .saveAsTable("us_delay_flights_tbl")

# Json

In [ ]:
val file = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/json/*"
val df = spark.read.format("json").load(file)

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

In [ ]:
df.write.json("C:/tmp/data/json/df_json2")

# csv

In [ ]:
val file = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/csv/*"
val schema = "DEST_COUNTRY_NAME STRING, ORIGIN_COUNTRY_NAME STRING, count INT"
val df = spark.read.format("csv")
 .schema(schema)
 .option("header", "true")
 .option("mode", "FAILFAST") // Exit if any errors
 .option("nullValue", "") // Replace any null data with quotes
 .load(file)

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

In [ ]:
df.write.format("csv").mode("overwrite").save("/tmp/data/csv/df_csv")

# avro

In [ ]:
val df = spark.read.format("avro")
.load("C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/avro/*")
df.show(false)

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show(false)

In [ ]:
df.write
 .format("avro")
 .mode("overwrite")
 .save("/tmp/data/avro/df_avro")

# orc

In [ ]:
val file = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/summary-data/orc/*"
val df = spark.read.format("orc").load(file)
df.show(10, false)

In [ ]:
spark.sql("SELECT * FROM us_delay_flights_tbl").show()

In [ ]:
df.write.format("orc")
 .mode("overwrite")
 .option("compression", "snappy")
 .save("/tmp/data/orc/df_orc")

# Image

In [ ]:
import org.apache.spark.ml.source.image
val imageDir = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val imagesDF = spark.read.format("image").load(imageDir)
imagesDF.printSchema
imagesDF.select("image.height", "image.width", "image.nChannels", "image.mode",
 "label").show(5, false)

# Binary files

In [ ]:
val path = "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/cctvVideos/train_images/"
val binaryFilesDF = spark.read.format("binaryFile")
 .option("pathGlobFilter", "*.jpg")
 .load(path)
binaryFilesDF.show(5)

In [ ]:
val binaryFilesDF = spark.read.format("binaryFile")
 .option("pathGlobFilter", "*.jpg")
 .option("recursiveFileLookup", "true")
 .load(path)
binaryFilesDF.show(5)

# Ejercicios capítulo 4

GlobalTempView vs TempView

GlobalTempView es una temp view que se puede utilizar en todas las sesiones de Spark, mientras que TempView solo se puede usar en una sesion

Leer los AVRO, Parquet, JSON y CSV escritos en el cap3

# Capítulo 5

In [ ]:
val cubed = (s: Long) => {
 s * s * s
}
// Register UDF
spark.udf.register("cubed", cubed)
// Create temporary view
spark.range(1, 9).createOrReplaceTempView("udf_test")
spark.sql("SELECT id, cubed(id) AS id_cubed FROM udf_test").show()

In [ ]:
val jdbcDF1 = spark
 .read
 .format("jdbc")
 .option("url", "jdbc:postgresql:[DBSERVER]")
 .option("dbtable", "[SCHEMA].[TABLENAME]")
 .option("user", "[USERNAME]")
 .option("password", "[PASSWORD]")
 .load()

# Higher-Order Functions in DataFrames and Spark SQL

In [ ]:
val t1 = Array(35, 36, 32, 30, 40, 42, 38)
val t2 = Array(31, 32, 34, 55, 56)
val tC = Seq(t1, t2).toDF("celsius")
tC.createOrReplaceTempView("tC")
// Show the DataFrame
tC.show()

In [ ]:
spark.sql("""
SELECT celsius,
transform(celsius, t -> ((t * 9) div 5) + 32) as fahrenheit 
 FROM tC
""").show()

In [ ]:
spark.sql("""
SELECT celsius, 
 filter(celsius, t -> t > 38) as high 
 FROM tC
""").show()

In [ ]:
//is there a temperature = 38
spark.sql("""
SELECT celsius, 
 exists(celsius, t -> t = 38) as threshold
 FROM tC
""").show()

In [ ]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{functions => F}
// Calculate average temperature and convert to F
spark.sql("""
SELECT celsius, 
 reduce(celsius, 0, 
 (t, acc) -> t + acc, 
 acc -> (acc div size(celsius) * 9 div 5) + 32) as avgFahrenheit 
 FROM tC
""").show()

In [ ]:
import org.apache.spark.sql.functions._
// Set file paths
val delaysPath =
 "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
val airportsPath =
 "C:/Users/xabier.jimenez/Documents/LearningSparkV2-master/databricks-datasets/learning-spark-v2/flights/airport-codes-na.txt"
val airports = spark.read
 .option("header", "true")
 .option("inferschema", "true")
 .option("delimiter", "\t")
 .csv(airportsPath)
airports.createOrReplaceTempView("airports_na")
// Obtain departure Delays data set
val delays = spark.read
 .option("header","true")
 .csv(delaysPath)
 .withColumn("delay", expr("CAST(delay as INT) as delay"))
 .withColumn("distance", expr("CAST(distance as INT) as distance"))
delays.createOrReplaceTempView("departureDelays")

val foo = delays.filter(
 expr("""origin == 'SEA' AND destination == 'SFO' AND 
 date like '01010%' AND delay > 0"""))
foo.createOrReplaceTempView("foo")

In [ ]:
spark.sql("SELECT * FROM airports_na LIMIT 10").show()

In [ ]:
spark.sql("SELECT * FROM departureDelays LIMIT 10").show()

In [ ]:
spark.sql("SELECT * FROM foo").show()

In [ ]:
val bar = delays.union(foo)
bar.createOrReplaceTempView("bar")
bar.filter(expr("""origin == 'SEA' AND destination == 'SFO' AND date LIKE '01010%' AND delay > 0""")).show()

In [ ]:
foo.join(
 airports.as('air),
 $"air.IATA" === $"origin"
).select("City", "State", "date", "delay", "distance", "destination").show()

In [ ]:
import org.apache.spark.sql.functions.expr
val foo2 = foo.withColumn(
 "status",
 expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END")
 )
foo2.show()

In [ ]:
val foo3 = foo2.drop("delay")
foo3.show()

In [ ]:
val foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

# Ejercicios capítulo 5

Pros y Cons utilizar UDFs

In [ ]:
Pros: puedes reutilizar codigo
Contras: es más lento

Instalar MySQL, descargar driver y cargar datos de BBDD de empleados 
https://dev.mysql.com/doc/employee/en/
i. Cargar con spark datos de empleados y departamento

In [9]:
val employees = spark
.read
.format("jdbc")
.option("url", "jdbc:mysql://localhost:3306/employees")
.option("driver", "com.mysql.jdbc.Driver")
.option("dbtable", "employees")
.option("user", "root")
.option("password", "jjzSx9PJ")
.load()

employees: org.apache.spark.sql.DataFrame = [emp_no: int, birth_date: date ... 4 more fields]


In [12]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [6]:
spark.sql("use employees;")
spark.sql("select A.*, B.salary, C.title from employees A join salaries B on A.emp_no=B.emp_no join titles C on A.emp_no=C.emp_no;").show()

org.apache.spark.sql.catalyst.analysis.NoSuchDatabaseException:  Database 'employees' not found